In [1]:
import findspark
findspark.init('/home/hadoop/spark-2.2.2-bin-hadoop2.7')
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
conf = SparkConf().setMaster("spark://master:7077")
sc = SparkContext.getOrCreate()
sqlcontext = SQLContext(sc)

import time

In [2]:
token_count=sqlcontext.read.parquet('hdfs://master:9000/user/hadoop/token_count')

In [3]:
#start = time.time()

#token_count_sum=token_count.groupBy('token').sum('count')
#token_count_sum.show(5)

#print('time :', time.time() - start)

In [4]:
from pyspark.sql.types import *
token_df = sqlcontext.read.parquet("hdfs://master:9000/user/hadoop/all_hangul_tokens")
token_df.select('title_id','tokens').show(5) #example\
token_df.count()

+--------+--------------------+
|title_id|              tokens|
+--------+--------------------+
|  112931|[정주행, 하시는, 분들, 추천...|
|  112931|[만, 렙이, 레이, 드, 못,...|
|  112931|[아랑소드, 용사, 시절, 에피...|
|  112931|[본격, 드래곤, 이, 자기, ...|
|  112931|[본격, 세계, 를구한, 드래곤...|
+--------+--------------------+
only showing top 5 rows



2586400

In [5]:
token_df.show(5)

+--------+-----------+-----------+--------------------+-------------------+
|title_id|episode_num|comment_num|              tokens|    registered_time|
+--------+-----------+-----------+--------------------+-------------------+
|  112931|          0|          0|[정주행, 하시는, 분들, 추천...|2013-05-22 01:07:04|
|  112931|          0|          1|[만, 렙이, 레이, 드, 못,...|2013-04-15 01:47:32|
|  112931|          0|          2|[아랑소드, 용사, 시절, 에피...|2013-05-02 08:37:28|
|  112931|          0|          3|[본격, 드래곤, 이, 자기, ...|2013-04-19 06:56:25|
|  112931|          0|          4|[본격, 세계, 를구한, 드래곤...|2013-04-26 07:52:28|
+--------+-----------+-----------+--------------------+-------------------+
only showing top 5 rows



In [6]:
import pyspark.sql.functions as F
split_udf=F.udf(lambda s: s[1:-1])
temp = token_df.select('title_id','tokens').withColumn('split', F.split(split_udf(token_df.tokens),', ')).select('title_id','split')

In [7]:

#split_udf=F.udf(lambda x : ''.join((x[1:-1]).replace(" ", "")).split(","))
#token_df=token_df.select('title_id','tokens').withColumn('split', split_udf(token_df.tokens))

split_udf=F.udf(lambda s: s[1:-1])
token_df=token_df.select('title_id','tokens').withColumn('split', F.split(split_udf(token_df.tokens),', ') )

token_df.show(5)
#mapped_df=token_df.select(token_df.title_id, F.explode(token_df.split).alias('token'), F.lit(1)).filter(F.length('token')>1)
#mapped_df.show(5)

+--------+--------------------+--------------------+
|title_id|              tokens|               split|
+--------+--------------------+--------------------+
|  112931|[정주행, 하시는, 분들, 추천...|[정주행, 하시는, 분들, 추천...|
|  112931|[만, 렙이, 레이, 드, 못,...|[만, 렙이, 레이, 드, 못,...|
|  112931|[아랑소드, 용사, 시절, 에피...|[아랑소드, 용사, 시절, 에피...|
|  112931|[본격, 드래곤, 이, 자기, ...|[본격, 드래곤, 이, 자기, ...|
|  112931|[본격, 세계, 를구한, 드래곤...|[본격, 세계, 를구한, 드래곤...|
+--------+--------------------+--------------------+
only showing top 5 rows



In [9]:
from pyspark.sql.functions import concat, col, lit

temp_tokendf = token_df.select("split").take(5)

split_df = token_df.select("title_id", "split")
split_df = split_df.limit(10000)


#a = token_df.rdd.map(lambda x: x.split(",")).toDF()



#print(type(temp_tokendf))
#print(temp_tokendf)

#for i in temp_tokendf:
#    print(type(i))
#    print(''.join((i['tokens'][1:-1]).replace(" ", "")).split(","))

    
#temp_tokendf = token_df.select('tokens').apply(lambda x : ''.join((x[1:-1]).replace(" ", "")).split(",")).toDF()


#token_df.select(col("tokens"))

token_df.printSchema()

#token_df.select("split").collect()
split_df.show(10)

root
 |-- title_id: integer (nullable = true)
 |-- tokens: string (nullable = true)
 |-- split: array (nullable = true)
 |    |-- element: string (containsNull = true)

+--------+--------------------+
|title_id|               split|
+--------+--------------------+
|  112931|[정주행, 하시는, 분들, 추천...|
|  112931|[만, 렙이, 레이, 드, 못,...|
|  112931|[아랑소드, 용사, 시절, 에피...|
|  112931|[본격, 드래곤, 이, 자기, ...|
|  112931|[본격, 세계, 를구한, 드래곤...|
|  112931|         [이거, 명작, 임]|
|  112931|[본격, 나런급의, 꿈도, 희망...|
|  112931|[이, 만화, 가, 그렇게, 멘...|
|  112931|[만렙, 찍고, 나면, 초급마을...|
|  112931|[엄청, 난, 명작이, 지만, ...|
+--------+--------------------+
only showing top 10 rows



In [10]:
start = time.time()

from pyspark.ml.feature import Word2Vec
#embedding_model = Word2Vec(temp_tokendf, size=100, window = 2, min_count=1, workers=4, iter=100, sg=1)



word2Vec = Word2Vec(vectorSize=100, minCount=0, inputCol="split", outputCol="result")


#word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="text", outputCol="result")
model = word2Vec.fit(split_df)

print('time :', time.time() - start)

model
#result = model.transform(documentDF)
#for row in result.collect():
#    text, vector = row
#    print("Text: [%s] => \nVector: %s\n" % (", ".join(text), str(vector)))

time : 14.921932220458984


Word2Vec_4d36a8c4993747f02229

In [11]:
wordVectorsDF = model.getVectors()
wordVectorsDF.show()

+------+--------------------+
|  word|              vector|
+------+--------------------+
|   선우가|[0.00404577329754...|
|    받음|[0.00537162646651...|
|   훈훈한|[0.00361029850319...|
|    쿤씨|[-0.0059975041076...|
|  안이뤄지|[0.00214941264130...|
|    근헤|[0.00117842713370...|
|   좋았던|[0.01105631049722...|
|보냈었는데요|[-0.0078351572155...|
| 작성해보았|[0.01037278026342...|
|    안한|[0.00444051902741...|
|   주연이|[-0.0035231341607...|
|    사진|[-0.0106863882392...|
|  아그네스|[0.00708514777943...|
|   톱질해|[2.21347960177809...|
| 치욕스러운|[-0.0021920048166...|
|    암시|[0.03119258582592...|
|     스|[-0.0781821087002...|
|들볶고싶구나|[-0.0021493446547...|
|    마이|[0.00452623143792...|
|   속세로|[-0.0052055893465...|
+------+--------------------+
only showing top 20 rows



In [12]:
model.explainParams()

''

In [13]:
model.findSynonyms('받음',2).show()

+----+------------------+
|word|        similarity|
+----+------------------+
|  물어|0.6366880536079407|
| 해주는|0.6241371631622314|
+----+------------------+



In [14]:
wordVectorsDF.take(1)

[Row(word='선우가', vector=DenseVector([0.004, -0.0012, -0.0021, 0.0047, -0.0043, -0.0035, 0.0027, 0.0034, 0.0012, 0.0021, 0.0036, -0.0051, -0.0045, 0.0048, 0.0009, -0.0012, 0.0046, -0.0001, -0.0016, 0.0031, -0.0041, -0.0025, 0.0016, 0.0013, 0.0006, -0.006, -0.002, -0.0006, 0.0006, 0.0013, -0.0017, -0.0051, 0.0058, 0.0029, -0.003, -0.0019, -0.0008, -0.0008, -0.0031, -0.0061, -0.0023, -0.0047, 0.0006, 0.0055, 0.0009, -0.0015, 0.0035, -0.0014, 0.0032, 0.0017, 0.0016, 0.0021, 0.0033, 0.0031, -0.0038, -0.0004, 0.0046, -0.0008, -0.0025, 0.0049, -0.0026, 0.0036, -0.0037, -0.0007, -0.0004, -0.0053, 0.0019, 0.0052, -0.0003, -0.0053, -0.0023, -0.0015, -0.0001, 0.0037, 0.0079, 0.0043, -0.0039, 0.0004, -0.0039, -0.006, -0.0002, 0.001, 0.0002, 0.0002, -0.0003, 0.0074, -0.0038, 0.0035, -0.0005, 0.0013, -0.0036, -0.0057, -0.005, 0.0079, -0.0091, 0.0002, -0.0017, 0.0006, -0.0043, -0.0042]))]

In [15]:
doc2vecs_df = model.transform(split_df)

In [16]:
doc2vecs_df.show()

+--------+--------------------+--------------------+
|title_id|               split|              result|
+--------+--------------------+--------------------+
|  112931|[정주행, 하시는, 분들, 추천...|[0.01058779986245...|
|  112931|[만, 렙이, 레이, 드, 못,...|[0.01010942008320...|
|  112931|[아랑소드, 용사, 시절, 에피...|[-0.0267542001092...|
|  112931|[본격, 드래곤, 이, 자기, ...|[0.00634275979828...|
|  112931|[본격, 세계, 를구한, 드래곤...|[-0.0010553386739...|
|  112931|         [이거, 명작, 임]|[0.03141902418186...|
|  112931|[본격, 나런급의, 꿈도, 희망...|[0.01205064291467...|
|  112931|[이, 만화, 가, 그렇게, 멘...|[0.02627593645145...|
|  112931|[만렙, 찍고, 나면, 초급마을...|[0.00642088868854...|
|  112931|[엄청, 난, 명작이, 지만, ...|[0.01919694915515...|
|  112931|[어떤, 용사, 의, 초건빵포,...|[-1.3217547287543...|
|  112931|         [멘붕, 의, 시작]|[0.03369739154974...|
|  112931|[이거, 진짜, 명작, 임, 방...|[0.01971330972238...|
|  112931| [이거, 명작, 임, 내가, 봤음]|[0.01982798180542...|
|  112931|   [드레곤, 세상을, 구, 했어]|[0.01291798986494...|
|  112931|[유료화, 하는, 게, 작가님,...|[0.013677535872

In [18]:
w2v_train_df, w2v_test_df = doc2vecs_df.randomSplit([0.8, 0.2])

In [20]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

si = StringIndexer(inputCol="title_id", outputCol="label")
rf_classifier = RandomForestClassifier(labelCol="label", featuresCol="result")

rf_classifier_pipeline = Pipeline(stages=[si,rf_classifier])
rf_predictions = rf_classifier_pipeline.fit(w2v_train_df).transform(w2v_test_df)

rf_model_evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")

In [21]:
accuracy = rf_model_evaluator.evaluate(rf_predictions)
print("Accuracy = %g" % (accuracy))

Accuracy = 0.692587
